In [398]:
import numpy as np
import pandas as pd
import utils.metropolishastings as mh
from utils.state import State
import random
from joblib import Parallel, delayed, cpu_count
from scipy.stats import uniform
import random
import math as m
import seaborn as sns
import copy
from utils.metropolishastings import generate_draws
import warnings
import os
import time
warnings.filterwarnings("ignore") 
import datetime
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Variable, Beta
import biogeme.database as db

In [399]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Functions needed to draw from conditionals

In [400]:
#Function for drawing a value using inverse transform for discrete variables
def discrete_inverse_trans(prob_vec):
    U=uniform.rvs(size=1)
    try:
        if U<=prob_vec[0]:
            return 1
        else:
            for i in range(1,len(prob_vec)+1):
                if sum(prob_vec[0:i])<U and sum(prob_vec[0:i+1])>U:
                    return i+1
    except:
        return 1

In [401]:
#Creating a probability vector from the choosen row/column from contigency matrix
def transform_into_marginals_clean(row):
    marginals= []
 
    suma = sum(row)
    for i in range(len(row)):
        marginals.append((row[i]/suma))
        
    return marginals

In [402]:
def draw_from_marginals(df, column):
    temp = df[column].value_counts().sort_index()
    index = discrete_inverse_trans(transform_into_marginals_clean(temp.tolist()))
    unique = sorted(temp.index)
    return unique[index-1]

In [403]:
#in order to form conditionals between different variables, we have to count each category of each variable
#because of this we have to create contigency tables
def create2dconditional(column1,column2):
    data_crosstab =  pd.crosstab(column1,column2, margins = False)
    return data_crosstab

def create3dconditional(column1, column2, column3):
    data_crosstab = pd.crosstab([column1,column2],[column3], margins = False)
    return data_crosstab  

def create4dconditional(column1, column2, column3, column4):
    data_crosstab = pd.crosstab([column1,column2,column3],[column4], margins = False)
    return data_crosstab  

def createNconditional(columns):
    data_crosstab = pd.crosstab(columns[:-1],columns[-1], margins = False)
    return data_crosstab

### Forming of conditionals from the data

In [404]:
path = "results" 
data_hh = 'data/synthetic_households.csv'
data_indiv = 'data/synthetic_individuals.csv'

# Create directory if not exists
if not os.path.exists(path):
    os.makedirs(path)

In [405]:
hh = pd.read_csv(data_hh)
individual_characteristics = pd.read_csv(data_indiv)

In [406]:
hh

,hid,htype,nbcars_agg,hsize,age_discrete,gender,marital_status,employment,driving_licence
0,0,10,1,1,5,2,4,4,1
1,1,10,1,1,5,2,1,4,1
2,2,10,1,1,5,2,1,4,1
3,3,10,1,1,5,2,1,4,1
4,4,10,1,1,5,2,1,4,1
...,...,...,...,...,...,...,...,...,...
124979,56770,220,2,3,4,2,2,1,1
124980,56770,220,2,3,1,2,1,5,2
124981,56771,220,2,3,4,1,2,1,1
124982,56771,220,2,3,4,2,2,1,1


### Helper functions because we don't have all info we need from the real data

In [407]:
def draw_age_from_label(age_label):
    """
    Given an age group label (1 to 7), return a random integer age from the corresponding range.
    """
    age_ranges = {
        1: range(0, 14),
        2: range(14, 18),
        3: range(18, 24),
        4: range(24, 44),
        5: range(44, 65),
        6: range(65, 94),
        7: range(94, 101),  # upper cap assumed
    }

    return random.choice(age_ranges.get(age_label, range(0, 101)))

hh['age'] = hh['age_discrete'].apply(draw_age_from_label)

In [408]:
def assign_role(house, arr):
  
    n = len(house)
    temp = house.sort_values(['age','gender'], ascending = [False, True])
    temp['role'] = np.arange(1,n+1)
    temp['nind'] = n
    arr.append(temp)
    


hh_arr = []
hh.groupby(['hid']).apply(lambda df:assign_role(df, hh_arr))
hh = pd.concat(hh_arr, axis=0)

In [409]:
hh

,hid,htype,nbcars_agg,hsize,age_discrete,gender,marital_status,employment,driving_licence,age,role,nind
0,0,10,1,1,5,2,4,4,1,55,1,1
1,1,10,1,1,5,2,1,4,1,56,1,1
2,2,10,1,1,5,2,1,4,1,55,1,1
3,3,10,1,1,5,2,1,4,1,56,1,1
4,4,10,1,1,5,2,1,4,1,62,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
124978,56770,220,2,3,4,1,2,1,1,24,2,3
124980,56770,220,2,3,1,2,1,5,2,0,3,3
124981,56771,220,2,3,4,1,2,1,1,42,1,3
124982,56771,220,2,3,4,2,2,1,1,42,2,3


### Drawing htype

In [410]:
def conditional_for_type(hh):
    pivot_df = hh.pivot_table(index='hid', columns=hh.groupby('hid').cumcount()+1, values='age', aggfunc='first')

    pivot_df.columns = [f'age_{i}' for i in pivot_df.columns]

    result_df = pivot_df.merge(hh[['hid', 'hsize','htype']].drop_duplicates(), on='hid').drop_duplicates()
    
    return result_df

In [411]:
def age_category(a, age_th=[15, 18, 24, 44, 65]):
    """Returns the age category for a person."""
    if (a == 0):
        return 0
    elif a < age_th[0]:
        return 1
    for i in range(1, len(age_th)):
        if (a >= age_th[i-1]) & (a < age_th[i]):
            return i+1
    return len(age_th)+1

In [412]:
def model_estimation_for_htype(hh):
    database = db.Database('households', hh)
    hsize = Variable('hsize')
    age_1 = Variable('age_1_d')
    age_2 = Variable('age_2_d')
    age_3 = Variable('age_3_d')
    htype = Variable('htype')
  
    beta_hsize_210 = Beta('beta_hsize_210', 0, None, None, 0)
    beta_hsize_220 = Beta('beta_hsize_220', 0, None, None, 0)
    beta_hsize_30 = Beta('beta_hsize_30', 0, None, None, 0)
    beta_age_1 = Beta('beta_age_1', 0, None, None, 0)
    beta_age_2 = Beta('beta_age_2', 0, None, None, 0)
    beta_age_3 = Beta('beta_age_3', 0, None, None, 0)
  
    

    cte_210 = Beta('cte_210', 0, None, None, 0)
    cte_220 = Beta('cte_220', 0, None, None, 0)
    cte_30 = Beta('cte_30', 0, None, None, 0)
    cte_230 = Beta('cte_230', 0, None, None, 1)
    


    V_210 = beta_hsize_210 * hsize + beta_age_1 * age_1 + age_2 * beta_age_2 + age_3 * beta_age_3 + cte_210
    V_220 = beta_hsize_220 * hsize + beta_age_1 * age_1 + age_2 * beta_age_2 + age_3 * beta_age_3 + cte_220
    V_30  = beta_hsize_30  * hsize + beta_age_1 * age_1 + age_2 * beta_age_2 + age_3 * beta_age_3 + cte_30
    V_230 = 0
    
    
  
    V= {30: V_30, 220: V_220, 210: V_210, 230: V_230}
    logprob = models.loglogit(V, None, htype) 
    
    the_biogeme = bio.BIOGEME(database, logprob)
    the_biogeme.modelName = 'example_1'
    
    results = the_biogeme.estimate()
    pandas_results = results.getEstimatedParameters()
    betas = results.getBetaValues()
    
    return betas

In [413]:
def V_210(betas, hsize, age_1, age_2, age_3):
    return betas['beta_hsize_210'] * hsize + betas['beta_age_1'] * age_1 + age_2 * betas['beta_age_2'] +  age_3 * betas['beta_age_3'] + betas['cte_210']

def V_220(betas, hsize, age_1, age_2, age_3):
        return betas['beta_hsize_220'] * hsize +  betas['beta_age_1'] * age_1 + age_2 * betas['beta_age_2'] +  age_3 * betas['beta_age_3'] + betas['cte_220']

def V_30(betas, hsize, age_1, age_2, age_3):
    return betas['beta_hsize_30'] * hsize +betas['beta_age_1'] * age_1 + age_2 * betas['beta_age_2'] +  age_3 * betas['beta_age_3']  + betas['cte_30']

def V_230(betas, hsize, age_1, age_2, age_3):
    return 0

In [414]:
def denom(betas, hsize, age_1, age_2, age_3):
    return np.exp(V_210(betas, hsize, age_1, age_2, age_3)) + np.exp(V_220(betas, hsize, age_1, age_2, age_3)) + np.exp(V_30(betas, hsize, age_1, age_2, age_3)) + np.exp(V_230(betas, hsize, age_1, age_2, age_3))

In [415]:
def P_30(betas, hsize, age_1, age_2, age_3):
    return np.exp(V_30(betas, hsize, age_1, age_2, age_3)) / denom(betas,hsize, age_1, age_2, age_3)

def P_210(betas, hsize, age_1, age_2, age_3):
    return np.exp(V_210(betas, hsize, age_1, age_2, age_3)) / denom(betas, hsize, age_1, age_2, age_3)

def P_220(betas, hsize, age_1, age_2, age_3):
    return np.exp(V_220(betas, hsize, age_1, age_2, age_3)) / denom(betas, hsize, age_1, age_2, age_3)

def P_230(betas, hsize, age_1, age_2, age_3):
    return np.exp(V_230(betas, hsize, age_1, age_2, age_3)) / denom(betas, hsize, age_1, age_2, age_3)

### Drawing number of cars

In [416]:
df = hh

def add_gender_counts(df):
    grouped = df.groupby('hid')['gender'].value_counts().unstack(fill_value=0)
    grouped.rename(columns={1: 'total_num_males', 2: 'total_num_females'}, inplace=True)
    df = df.merge(grouped, left_on='hid', right_index=True, how='left')
    return df

# Add gender counts to the dataframe
df_with_counts = add_gender_counts(df)

In [417]:
def add_age_counts(df):
    grouped = df.groupby('hid')['age_discrete'].apply(lambda x: pd.cut(x, bins=[0, 17, float('inf')], labels=['num_children', 'num_adults'], right=False).value_counts())
    grouped = grouped.unstack(fill_value=0)
    
    df = df.merge(grouped, left_on='hid', right_index=True, how='left')
    return df

# Add age counts to the dataframe
df_with_counts = add_age_counts(df_with_counts)

In [418]:
def add_total_licence_counts(df):
    """
    Adds 'total_num_licences' column to each row of the DataFrame.
    It counts how many individuals in each household have a driving licence (== 1).
    """
    licence_counts = df.groupby('hid')['driving_licence'].apply(lambda x: (x == 1).sum())
    df = df.merge(licence_counts.rename('total_num_licences'), on='hid', how='left')
    return df

df_with_counts = add_total_licence_counts(df_with_counts)

In [419]:
#contains only households information
grouped = df_with_counts.groupby('hid').agg({
    'htype':'first',
    'hsize':'first',
    'nbcars_agg':'first',
    'total_num_licences': 'first',
    'num_children': 'first',
    'num_adults': 'first',
    'total_num_males': 'first',
    'total_num_females': 'first'
}).reset_index()

In [420]:
grouped

,hid,htype,hsize,nbcars_agg,total_num_licences,num_children,num_adults,total_num_males,total_num_females
0,0,10,1,1,1,1,0,0,1
1,1,10,1,1,1,1,0,0,1
2,2,10,1,1,1,1,0,0,1
3,3,10,1,1,1,1,0,0,1
4,4,10,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...
56767,56767,220,4,4,2,4,0,1,3
56768,56768,220,4,3,2,4,0,3,1
56769,56769,220,3,0,2,3,0,2,1
56770,56770,220,3,2,2,3,0,1,2


In [421]:
hh

,hid,htype,nbcars_agg,hsize,age_discrete,gender,marital_status,employment,driving_licence,age,role,nind
0,0,10,1,1,5,2,4,4,1,55,1,1
1,1,10,1,1,5,2,1,4,1,56,1,1
2,2,10,1,1,5,2,1,4,1,55,1,1
3,3,10,1,1,5,2,1,4,1,56,1,1
4,4,10,1,1,5,2,1,4,1,62,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
124978,56770,220,2,3,4,1,2,1,1,24,2,3
124980,56770,220,2,3,1,2,1,5,2,0,3,3
124981,56771,220,2,3,4,1,2,1,1,42,1,3
124982,56771,220,2,3,4,2,2,1,1,42,2,3


In [422]:
hh.to_csv("household_full.csv", index=False)

### Drawing driving licence

In [423]:
def conditional_for_driving(hsize, subset, order_ind, lista):
    
    pivot_df = subset.pivot_table(index='hid', columns=subset.groupby('hid').cumcount()+1, values='driving_licence', aggfunc='first')


    pivot_df.columns = [f'driving_licence{i}' for i in pivot_df.columns]

    result_df = pivot_df.merge(subset[['hid', 'nbcars_agg']].drop_duplicates(), on='hid').drop_duplicates()

    if(hsize==2):
        if(order_ind == 1):
            table = create3dconditional(result_df['nbcars_agg'].values, result_df['driving_licence2'].values, result_df['driving_licence1'].values) 
        else:
            table = create3dconditional(result_df['nbcars_agg'].values, result_df['driving_licence1'].values, result_df['driving_licence2'].values)
    else:
        table = create4dconditional(result_df['nbcars_agg'].values, result_df[f'driving_licence{lista[0]}'].values, result_df[f'driving_licence{lista[1]}'].values, result_df[f'driving_licence{lista[2]}'].values)

    return table

In [424]:
def round_to_zero(x, threshold=1e-5):
    if abs(x) < threshold:
        return 0
    else:
        return x

In [425]:
def sample_from_probabilities(probabilities):
    # Normalize the probabilities
    total_prob = sum(probabilities)
    normalized_probabilities = [p / total_prob for p in probabilities]

    # Generate a random number between 0 and 1
    random_num = random.random()

    # Calculate cumulative probabilities
    cumulative_probabilities = [sum(normalized_probabilities[:i+1]) for i in range(len(normalized_probabilities))]

    # Select a category
    for i, cum_prob in enumerate(cumulative_probabilities):
        if random_num <= cum_prob:
            return i 

### Generation per household size group

In [426]:
for hsize_ref in [1,2,3]:
    hh = pd.read_csv("data/household_full.csv")
    individual_characteristics = pd.read_csv(data_indiv)
    
    if hsize_ref >= 3:
        hh = hh[hh['hsize'] >= hsize_ref]
        hh = hh[~((hh['htype']==10)&(hh['age_discrete']<=2))] # delete child in single household
        age_dataset = conditional_for_type(hh)
        age_dataset['age_1_d'] = age_dataset['age_1'].apply(lambda x: age_category(x))
        age_dataset['age_2_d'] = age_dataset['age_2'].apply(lambda x: age_category(x))
        age_dataset['age_3_d'] = age_dataset['age_3'].apply(lambda x: age_category(x))
        age_dataset.fillna(0, inplace=True)
        age_dataset = age_dataset[['hid','age_1_d','age_2_d','age_3_d','hsize','htype']]
        betas = model_estimation_for_htype(age_dataset)
        individual_characteristics = individual_characteristics[individual_characteristics['hsize'] >= hsize_ref]
        grouped_size = grouped[grouped['hsize']>=hsize_ref]
    else:
        hh = hh[hh['hsize'] == hsize_ref]
        hh = hh[~((hh['htype']==10)&(hh['age_discrete']<=2))] # delete child in single household
        if(hsize_ref==2):
            age_dataset = conditional_for_type(hh)
            age_dataset['age_1_d'] = age_dataset['age_1'].apply(lambda x: age_category(x))
            age_dataset['age_2_d'] = age_dataset['age_2'].apply(lambda x: age_category(x))
            age_dataset.fillna(0, inplace=True)
            age_dataset['age_3_d'] = 0
            betas = model_estimation_for_htype(age_dataset)
            
        individual_characteristics = individual_characteristics[individual_characteristics['hsize'] == hsize_ref]
        grouped_size = grouped[grouped['hsize']==hsize_ref]

    
    sizes_all = hh.groupby('hid').hsize.first()
    hsize = sizes_all.value_counts().sort_index()/sizes_all.value_counts().sum()
       
  
    cond_driving_single = create3dconditional(hh['nbcars_agg'].values, hh['age_discrete'].values,hh['driving_licence'].values)
   
    cond_gender_single = create2dconditional(hh['htype'].values, hh['gender'].values)
    def draw_from_htype_b(hsize,people):
        '''
        Draw type from size
        '''
        if(hsize == 2):
            age_owner = people[0][0]
            age_second = people[1][0]
        else:
            age_owner = people[0][0]
            age_second = people[1][0]
            age_third = people[2][0]
            
        if(hsize==1):
            return 10
        else:
            if(hsize==2):
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_type.loc[age_owner].loc[age_second].tolist()))
                unique = sorted(cond_type.loc[age_owner].loc[age_second].index)
                return unique[index - 1]
            else:
                try:
                    index = discrete_inverse_trans(transform_into_marginals_clean(cond_type.loc[age_owner].loc[age_second].loc[age_third].tolist()))
                    unique = sorted(cond_type.loc[age_owner].loc[age_second].loc[age_third].index)
                    return unique[index - 1]
                except:
                    return 220
                
    def draw_type_model(hsize, people):
        if(hsize==1):
            return 10
        elif(hsize == 2):
            age_1 = people[0][0]
            age_2 = people[1][0]
            age_3 = 0   
        else:
            age_1 = people[0][0]
            age_2 = people[1][0]
            age_3 = people[2][0]

        niz = [round_to_zero(P_30(betas, hsize, age_1, age_2, age_3)),round_to_zero(P_210(betas, hsize, age_1, age_2, age_3)), round_to_zero(P_220(betas, hsize, age_1, age_2, age_3)), round_to_zero(P_230(betas, hsize, age_1, age_2, age_3))]
      
        index = sample_from_probabilities(niz)
        unique = [30, 210, 220, 230]
        return unique[index]
                
    def draw_driving_owner(nbcars, age, size, licences):
        if age <= 2: #if young (exist single househould with age 2)
            return 2
        
        if(size==1):
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving_single.loc[nbcars].loc[age].tolist()))
                unique = sorted(cond_driving_single.loc[nbcars].loc[age].index)
                return unique[index-1]
            except:
                return 2
        elif(size==2):
            try:
                lista=[]
                cond_driving = conditional_for_driving(size,hh,1,lista)
                licence1 = licences[0]
                licence2 = licences[1]
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving.loc[nbcars].loc[licence2].tolist()))
                unique = sorted(cond_driving.loc[nbcars].loc[licence2].index)
                #print(f'Drawing for nb cars:{nbcars} for owner age: {age}, chosen licence: {unique[index-1]}')
                return unique[index - 1]
            except:
                return 2
        else:#ovo privremeno
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving_single.loc[nbcars].loc[age].tolist()))
                unique = sorted(cond_driving_single.loc[nbcars].loc[age].index)
                #print(f'Drawing for nb cars:{nbcars} for owner age: {age}, chosen licence: {unique[index-1]}')
                return unique[index-1]
            except:
                return 2
    
    def draw_driving_spouse(nbcars, age, size, licences):
        if age <= 2: #if young (exist single househould with age 2)
            return 2
        
        if(size==2):
            lista=[]
            cond_driving = conditional_for_driving(size,hh,2,lista)
            licence1 = licences[0]
            licence2 = licences[1]
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving.loc[nbcars].loc[licence1].tolist()))
                unique = sorted(cond_driving.loc[nbcars].loc[licence1].index)
                return unique[index - 1]
            except:
                return 2
        else:
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving.loc[age].loc[nbcars].tolist()))
                unique = sorted(cond_driving.loc[age].loc[nbcars].index)  
                return unique[index-1]    
            except:
                subset = individual_characteristics[individual_characteristics['age_discrete'] == age]
                index = draw_from_marginals(subset,'driving_licence')
                return index
            
    def draw_driving_first_and_second(nbcars, age, size, licences,role):
      
        if age <= 2: #if young (exist single househould with age 2)
            return 2
        
        if(size==1):
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving_single.loc[nbcars].loc[age].tolist()))
                unique = sorted(cond_driving_single.loc[nbcars].loc[age].index)
                return unique[index-1]
            except:
                return 2
        else:
            lista=[]
            cond_driving = conditional_for_driving(size,hh,role,lista)
            licence1 = licences[0]
            licence2 = licences[1]
            if(role==1):
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving.loc[nbcars].loc[licence2].tolist()))
                unique = sorted(cond_driving.loc[nbcars].loc[licence2].index)
                return unique[index - 1]
            elif(role==2):
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving.loc[nbcars].loc[licence1].tolist()))
                unique = sorted(cond_driving.loc[nbcars].loc[licence1].index)
                return unique[index - 1]        
        
    

    def draw_driving(age, driving, htype, nbcars, licences):
        '''
        Driving license conditioned on age and nbcars
        '''
        if age <= 2: #if young (exist single househould with age 2)
            return 2
        
        if (nbcars >= 1) and (driving == 1) and (licences == 1): #if there is cars in the household but no others have driving licence
            return 1
        
        else:
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_driving.loc[age].loc[nbcars].tolist()))
                unique = sorted(cond_driving.loc[age].loc[nbcars].index)  
                return unique[index-1]    
            except:
                subset = individual_characteristics[individual_characteristics['age_discrete'] == age]
                index = draw_from_marginals(subset,'driving_licence')
                return index
            
    def draw_from_nbcars_b(htype_value, hsize_value, licences):
         try:
            index = discrete_inverse_trans(transform_into_marginals_clean(grouped_size['nbcars_agg'].value_counts().sort_index()))
            unique = grouped_size['nbcars_agg'].value_counts().sort_index().index
            return unique[index - 1]
         except:
            if (licences>0):
                return 2
            else:
                return 1
    
  
    
    house = hh[hh['hsize']>=3]
    cond_age_owner = create4dconditional(house[house['role']==1]['htype'].values,house[house['role'] == 2]['age_discrete'].values,\
                                        house[house['role'] == 3]['age_discrete'].values, house[house['role'] == 1]['age_discrete'].values)

    house = hh[hh['hsize']>=2]
    cond_age_owner_1 = create3dconditional(house[house['role']==1]['htype'].values,house[house['role'] == 2]['age_discrete'].values,\
                                        house[house['role'] == 1]['age_discrete'].values)

    cond_age_owner_2 = create2dconditional(hh[hh['role']==1]['htype'].values, hh[hh['role'] == 1]['age_discrete'].values)

    cond_age_owner_3 = create2dconditional(house[house['role']==2]['age_discrete'].values, house[house['role'] == 1]['age_discrete'].values)

    individual = hh[hh['hsize']==1]
    cond_age_owner_full = createNconditional([individual['nbcars_agg'], individual['gender'], individual['marital_status'],\
                                            individual['employment'],individual['driving_licence'],individual['age_discrete']])



    # gender
    house = hh[hh['hsize']>=2]
    cond_gender_owner = create3dconditional(house[house['role']==1]['htype'].values,house[house['role'] == 2]['gender'].values,\
                                        house[house['role'] == 1]['gender'].values)

    cond_gender_owner_1 = create2dconditional(hh[hh['role']==1]['htype'].values,hh[hh['role'] == 1]['gender'].values)

    cond_gender_owner_full = createNconditional([individual['nbcars_agg'], individual['age_discrete'], individual['marital_status'],\
                                            individual['employment'],individual['driving_licence'],individual['gender']])


    # marital status
    owners = hh[hh['role']==1]
    cond_marital_owner = create3dconditional(owners['htype'].values, owners['age_discrete'].values,owners['marital_status'].values)
    cond_marital_owner_1 = create2dconditional(owners['htype'].values,owners['marital_status'].values)

    # employment/driving
    cond_employment = create2dconditional(hh['age_discrete'],hh['employment'])
    
    def draw_age_owner(htype, nbcars, gender, marital, employment, driving, age_second = None, age_member = None):
        '''
        Age conditioned on htype, age of the second and third individual
        '''
        try:
            if age_second and age_member:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_owner.loc[htype].loc[age_second].loc[age_member].tolist()))
                unique = sorted(cond_age_owner.loc[htype].loc[age_second].loc[age_member].index)  
                return unique[index-1]
            elif age_second:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_owner_1.loc[htype].loc[age_second].tolist()))
                unique = sorted(cond_age_owner_1.loc[htype].loc[age_second].index)  
                return unique[index-1]
            else:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_owner_full.loc[nbcars].loc[gender].loc[marital]\
                                                                            .loc[employment].loc[driving].tolist()))
                unique = sorted(cond_age_owner_full.loc[nbcars].loc[gender].loc[marital]\
                                                    .loc[employment].loc[driving].index)  
                return unique[index-1]
        except:
            if age_second:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_owner_3.loc[age_second].tolist()))
                unique = sorted(cond_age_owner_3.loc[age_second].index)  
                return unique[index-1]
            else:
                subset = individual_characteristics[individual_characteristics['hsize'] == 1]
                index = draw_from_marginals(subset,'age_discrete')
                return index
            
    def draw_gender_owner(htype,  nbcars, age, marital, employment, driving, gender_second = None):
        '''
        Gender of owner from htype and gender of the second
        '''
        try:
            if gender_second:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_gender_owner.loc[htype].loc[gender_second].tolist()))
                unique = sorted(cond_gender_owner.loc[htype].loc[gender_second].index)  
                return unique[index-1]
            else:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_gender_owner_full.loc[nbcars].loc[age].loc[marital]\
                                                                            .loc[employment].loc[driving].tolist()))
                unique = sorted(cond_gender_owner_full.loc[nbcars].loc[age].loc[marital]\
                                                    .loc[employment].loc[driving].index)  
                return unique[index-1]             
        except:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_gender_owner_1.loc[htype].tolist()))
                unique = sorted(cond_gender_owner_1.loc[htype].index)  
                return unique[index-1]
    
    def draw_marital_owner(htype, age_owner):
        '''
        Marital status conditioned on htype, age and gender
        '''
        if age_owner <= 2:
            return 1
        
        try:
            index = discrete_inverse_trans(transform_into_marginals_clean(cond_marital_owner.loc[htype].loc[age_owner].tolist()))
            unique = sorted(cond_marital_owner.loc[htype].loc[age_owner].index)  
            return unique[index-1]            
        except:
            
            index = discrete_inverse_trans(transform_into_marginals_clean(cond_marital_owner_1.loc[htype].tolist()))
            unique = sorted(cond_marital_owner_1.loc[htype].index)  
            return unique[index-1]          
                
    def draw_employment(age):
        '''
        Employment conditioned on age
        '''
        if age == 1:
            return 5
        elif age == 7:
            return 2
        else:
            index = discrete_inverse_trans(transform_into_marginals_clean(cond_employment.loc[age].tolist()))
            unique = sorted(cond_employment.loc[age].index)  
            return unique[index-1]  
   

    ### Attributes of the second person
    # conditionals
    # age
    house = hh[hh['hsize']>=3]
    cond_age_second = create4dconditional(house[house['role']==1]['htype'].values,house[house['role'] == 1]['age_discrete'].values,\
                                        house[house['role'] == 3]['age_discrete'].values, house[house['role'] == 2]['age_discrete'].values)

    house = hh[hh['hsize']>=2]
    cond_age_second_1 = create3dconditional(house[house['role']==1]['htype'].values,house[house['role'] == 1]['age_discrete'].values,\
                                        house[house['role'] ==2]['age_discrete'].values)

    cond_age_second_2 = create2dconditional(hh[hh['role']==2]['htype'].values, hh[hh['role'] == 2]['age_discrete'].values)

    # gender
    house = hh[hh['hsize']>=2]
    cond_gender_second = create3dconditional(house[house['role']==1]['htype'].values,house[house['role'] == 1]['gender'].values,\
                                        house[house['role'] == 2]['gender'].values)

    cond_gender_second_1 = create2dconditional(hh[hh['role']==2]['htype'].values,hh[hh['role'] == 2]['gender'].values)

    # marital status
    seconds = hh[hh['role'] == 2]
    cond_marital_second = create3dconditional(seconds['htype'].values, seconds['age_discrete'].values, seconds['marital_status'].values)
    cond_marital_second_1 = create2dconditional(seconds['htype'].values, seconds['marital_status'].values)

    def draw_age_second(htype, age_owner, age_member = None):
        '''
        Age conditioned on htype, age of the first and third individual
        '''
        try:
            if age_member:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_second.loc[htype].loc[age_owner].loc[age_member].tolist()))
                unique = sorted(cond_age_second.loc[htype].loc[age_owner].loc[age_member].index)  
                return unique[index-1]
            else:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_second_1.loc[htype].loc[age_owner].tolist()))
                unique = sorted(cond_age_second_1.loc[htype].loc[age_owner].index)  
                return unique[index-1]            
        except:
            if age_member:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_second_2.loc[htype].tolist()))
                unique = sorted(cond_age_second_2.loc[htype].index)  
                return unique[index-1]
    def draw_gender_second(htype, gender_owner):
        '''
        Gender of owner from htype and gender of the second
        '''
        try:
            index = discrete_inverse_trans(transform_into_marginals_clean(cond_gender_second.loc[htype].loc[gender_owner].tolist()))
            unique = sorted(cond_gender_second.loc[htype].loc[gender_owner].index)  
            return unique[index-1]           
        except:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_gender_second_1.loc[htype].tolist()))
                unique = sorted(cond_gender_second_1.loc[htype].index)  
                return unique[index-1]
    def draw_marital_second(htype, age_second):
        '''
        Marital status conditioned on htype, age and gender
        '''
        if age_second <= 2:
            return 1
        
        try:
            index = discrete_inverse_trans(transform_into_marginals_clean(cond_marital_second.loc[htype].loc[age_second].tolist()))
            unique = sorted(cond_marital_second.loc[htype].loc[age_second].index)  
            return unique[index-1]           
        except:
            
            index = discrete_inverse_trans(transform_into_marginals_clean(cond_marital_second_1.loc[htype].tolist()))
            unique = sorted(cond_marital_second_1.loc[htype].index)  
            return unique[index-1]          
    ### Attributes of other individuals (3rd, 4th, 5th, ...)
    # age
    house = hh[hh['hsize']>=3]
    cond_age_third = create4dconditional(house[house['role'] == 1]['htype'].values, house[house['role'] == 1]['age_discrete'].values,\
                                        house[house['role'] == 2]['age_discrete'].values,house[house['role'] == 3]['age_discrete'].values)

    # Marital status
    members = hh[hh['role'] >= 3]
    cond_marital_members = create3dconditional(members['htype'].values, members['age_discrete'].values, members['marital_status'].values)
    cond_marital_members_1 = create2dconditional(members['htype'].values, members['marital_status'].values)


    # conditionals of age of the next person on the previous one
    temp_type = []
    temp_pre = []
    temp_next = []
    for i in range(2,15):
        house = hh[hh['hsize']>=i]
        temp_type.append(house[house['role'] == 1]['htype'])
        temp_pre.append(house[house['role'] == i-1]['age_discrete'])
        temp_next.append(house[house['role'] == i]['age_discrete'])
        
    htype = pd.concat(temp_type[2:], axis=0).values
    hpre = pd.concat(temp_pre[2:], axis=0).values
    hnext = pd.concat(temp_next[2:], axis=0).values
    cond_age_member = create3dconditional(htype, hpre, hnext)

    htype = pd.concat(temp_type, axis=0).values
    hpre = pd.concat(temp_pre, axis=0).values
    hnext = pd.concat(temp_next, axis=0).values
    cond_age_member_1 = create3dconditional(htype, hpre, hnext)

    # gender
    house = hh[hh['hsize']>=3]
    cond_gender_member = create2dconditional(house[house['role'] >= 3]['htype'].values,house[house['role'] >= 3]['gender'].values)
    def draw_age_member(age_pre, htype, age_owner = None, age_second = None):
        '''
        If age_owner = None and age_second = None, it's the first child of the household
        '''
        try:
            if age_owner and age_second:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_third.loc[htype].loc[age_owner].loc[age_second].tolist()))
                unique = sorted(cond_age_third.loc[htype].loc[age_owner].loc[age_second].index)  
                return unique[index-1]
            else:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_member.loc[htype].loc[age_pre].tolist()))
                unique = sorted(cond_age_member.loc[htype].loc[age_pre].index)  
                return unique[index-1]            
        except:
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_age_member_1.loc[htype].loc[age_pre].tolist()))
                unique = sorted(cond_age_member_1.loc[htype].loc[age_pre].index)  
                return unique[index-1] 
            except:
                print(f"{htype} {age_pre}")
                return age_pre
   
    def draw_gender_member(htype):
        index = discrete_inverse_trans(transform_into_marginals_clean(cond_gender_member.loc[htype].tolist()))
        unique = sorted(cond_gender_member.loc[htype].index)  
        return unique[index-1] 
    def draw_marital_member(age_member, htype):
        
        if age_member<=2:
            return 1
        else:
            try:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_marital_members.loc[htype].loc[age_member].tolist()))
                unique = sorted(cond_marital_members.loc[htype].loc[age_member].index)  
                return unique[index-1]
            except:
                index = discrete_inverse_trans(transform_into_marginals_clean(cond_marital_members_1.loc[htype].tolist()))
                unique = sorted(cond_marital_members_1.loc[htype].index)  
                return unique[index-1]                     
    # Functions for individuals generation
    def generate_owner(agents,htype_value,nbcars,licences,licence_driving,hsize_value):
        
        X_curr = copy.deepcopy(agents[0])
        age_second = agents[1][0] if agents[1][0]!=0 else None
        age_member = agents[2][0] if agents[2][0]!=0 else None
        gender_second = agents[1][1] if agents[1][1]!=0 else None
            
        
        r = random.randint(0,4)
        if(r == 0):#age
            X_curr[0] = draw_age_owner(htype_value,nbcars, X_curr[1], X_curr[2], X_curr[3], X_curr[4], age_second, age_member)
            if X_curr[0] <= 2:
                X_curr[2] = 1
                X_curr[4] = 2
            if X_curr[0] == 1:
                X_curr[3] = 3

        elif (r == 1):#gender
            X_curr[1] = draw_gender_owner(htype_value, nbcars, X_curr[0], X_curr[2], X_curr[3], X_curr[4], gender_second)

        elif (r == 2):#marital status
            X_curr[2] = draw_marital_owner(htype_value, X_curr[0])

        elif (r == 3):#employment
            X_curr[3] = draw_employment(X_curr[0])

        else:#driving licence
         
            X_curr[4] = draw_driving_owner(nbcars, X_curr[0], hsize_value, licence_driving)
       

        return X_curr

    def generate_second(agents, htype, nbcars,licences,licence_driving, hsize_value):
        
        X_curr = copy.deepcopy(agents[1])
        age_owner = agents[0][0]
        gender_owner = agents[0][1]
        # marital_owner = agents[0][2]
        age_member = agents[2][0] if agents[2][0]!=0 else None
        #agent = [1,1,1,1,1]#[age, gender, marital status, employment, driving]
        

        # If the intial state of the second person is all zero, initialize first
        if X_curr[0] == 0:

            X_curr[0] = draw_age_second(htype,age_owner,age_member)
            X_curr[1] = draw_gender_second(htype, gender_owner)
            X_curr[2] = draw_marital_second(htype,X_curr[0])
            X_curr[3] = draw_employment(X_curr[0])
            X_curr[4] = draw_driving_spouse(nbcars, X_curr[0], hsize_value, licence_driving)

        r = random.randint(0,4)
        if(r == 0):#age
            X_curr[0] = draw_age_second(htype,age_owner,age_member)

            if X_curr[0] <= 2:
                X_curr[2] = 1
                X_curr[4] = 2
            if X_curr[0] == 1:
                X_curr[3] = 3     

        elif (r == 1):#gender
            X_curr[1] = draw_gender_second(htype, gender_owner)

        elif (r == 2):#marital status
            X_curr[2] = draw_marital_second(htype,X_curr[0])
            
        elif (r == 3):#employment
            X_curr[3] = draw_employment(X_curr[0])

        else:#driving licence
         
            X_curr[4] = draw_driving_spouse(nbcars, X_curr[0], hsize_value, licence_driving)
            
 
        return X_curr
    def generate_member(agents, htype, nbcars, licences, i, licence_driving,hsize_value,role):
        '''
        previous_agents: list of all previous agents
        i: index of current agent
        '''
        X_curr = copy.deepcopy(agents[i])
        age_pre = agents[i-1][0]
        age_owner = agents[0][0] if i==3 else None
        age_second = agents[1][0] if i==3 else None
        #agent = [1,1,1,1,1]#[age, gender, marital status, employment, driving]
        
        # If the intial state of the second person is all zero, initialize first
        if X_curr[0] == 0:

            X_curr[0] = draw_age_member(age_pre, htype, age_owner, age_second)
            X_curr[1] = draw_gender_member(htype)
            X_curr[2] = draw_marital_member(X_curr[0],htype)
            X_curr[3] = draw_employment(X_curr[0])
            X_curr[4] = draw_driving(X_curr[0], X_curr[4], htype, nbcars, licences)

        r = random.randint(0,4)
        if(r == 0):#age
            X_curr[0] = draw_age_member(age_pre, htype, age_owner, age_second)

            if X_curr[0] <= 2:
                X_curr[2] = 1
                X_curr[4] = 2
            if X_curr[0] == 1:
                X_curr[3] = 3      

        elif (r == 1):#gender
            X_curr[1] = draw_gender_member(htype)

        elif (r == 2):#marital status
            X_curr[2] = draw_marital_member(X_curr[0],htype)

        elif (r == 3):#employment
            X_curr[3] = draw_employment(X_curr[0])

        else:#driving licence
           
            X_curr[4] = draw_driving(X_curr[0], X_curr[4], htype, nbcars, licences)
        
            
 
        return X_curr
    def draw_feature_people(list_of_people, htype, nbcars, licences, i,licence_driving,hsize_value):
        '''
        Change one feature in agent
        '''
        if i == 0:#owner:
            temp = generate_owner(list_of_people,htype,nbcars,licences,licence_driving,hsize_value)
        elif i == 1:# second person
            temp = generate_second(list_of_people,htype,nbcars,licences,licence_driving,hsize_value)
        else:# member
            temp = generate_member(list_of_people,htype,nbcars,licences,i,licence_driving,hsize_value,i)
            
        result = copy.deepcopy(list_of_people)
        result[i] = temp
        
        return result
    # Gibbs sampler generator
    class MetropolisHastingsIterator:
        """Implements the Markov chain for the MH algorithm"""

        def __init__(self, initialState):
            """Constructor

            :param initialState: the initial state of the Markov processe
            :type initialState: State

            :param userProcess: function that takes a state as input, and
                returns a new state, the forward and the backward
                transition probabilities.
            :type userProcess: state, pij, pji = fct(state)

            :param logweight: log of the unnormalized target sampling
                probability. Function that takes a state as input.
            :type logweight: float = fct(state)

            """

            self.currentState = initialState
            self.accepted = 0
            self.rejected = 0
            self.sequence = np.array([])

        def __iter__(self):
            """As the object is both an iterable and an iterator, it returns
            itself.
            """
            return self

        def getSuccessRate(self):
            """Computes the percentage of accepted draws

            :return: percentage of accepted draws
            :rtype: float
            """
            total = self.accepted + self.rejected
            if total == 0:
                return 0
            return float(self.accepted) / float(total)

        def __next__(self):
            """Generate the next state of the Markov process"""
        
            candidate, logqij, logqji = self.currentState.next_state()
            logwi = self.currentState.logweight()
            logwj = candidate.logweight()
            logratio = logwj + logqji - logwi - logqij
            logalpha_ij = min(logratio, 0)
            r = np.random.uniform()
            if np.log(r) < logalpha_ij:
                self.currentState = candidate
                self.accepted += 1
            else:
                self.rejected += 1
            return self.currentState


    class AutoCorrelation:
        """Calculates the autocorrelations defined by formula (11.7)
        in Gelman et al.
        """

        def __init__(self, draws, var):
            """Constructor

            :param draws: m arrays or n draws
            :type draws: np.array([np.array(float)])

            :param var: estimate of the posterio variance, defined by (11.3)
            :type var: float

            """
            self.m, self.n = draws.shape
            self.draws = draws
            self.var = var
            self.reset()

        def reset(self):
            """Initialize the variables at the beginning of each loop."""
            self.t = 1
            self.last_rho = None
            self.negative_autocorrelation = False

        def __iter__(self):
            """As the object is both an iterable and an iterator, it returns
            itself.
            """
            self.reset()
            return self

        def variogram(self):
            """Calculates the variogram on p. 286 for the current value of t"""
            return (
                (self.draws[:, self.t :] - self.draws[:, : (self.n - self.t)]) ** 2
            ).sum() / (self.m * (self.n - self.t))

        def __next__(self):
            """Implements one iteration

            :return: current autocorrelation
            :rtype: float

            :raise StopIteration: when the list is exhausted or negative
                autocorrelation has been detected

            """
            if self.negative_autocorrelation:
                raise StopIteration
            if self.t >= self.n:
                raise StopIteration

            rho = 1 - self.variogram() / (2 * self.var)
            if not self.t % 2:
                if rho + self.last_rho < 0:
                    self.negative_autocorrelation = True
            self.last_rho = rho
            self.t += 1
            return rho


    def AnalyzeDraws(draws):
        """Calculates the potential scale reduction and the effective number
        of simulation draws. See Gelman et al. Chapter 11.

        :param draws: S x R array of draws, where S is the number of
            sequences, and R the number of draws per sequence.

        :type draws: numpy.array

        :return: potential scale reduction, effective numner of draws and
            mean of the indicators
        :rtype: float, int, numpy.array(float)
        """

        nbrOfSequences, nbrOfDraws = draws.shape
        m = 2 * nbrOfSequences
        n = int(nbrOfDraws / 2)
        draws = draws.reshape(m, n)

        # The name of the variables below refer to the notations in
        # Gelman et al.

        # Means
        phi_bar_dot_j = [np.mean(d) for d in draws]
        phi_bar = np.mean(phi_bar_dot_j)
        B = np.var(phi_bar_dot_j, ddof=1) * n

        # Variances. ddof=1 means that we divide by n-1 and not by
        # n. See numpy documentation
        s_j_squared = [np.var(d, ddof=1) for d in draws]
        W = np.mean(s_j_squared)

        # Calculation of the marginal posterior variance (11.3) and
        # the potential scale reduction (11.4)
        var_plus = (n - 1) * W / n + B / n
        R_hat = np.sqrt(var_plus / W)

        # Calculation of the effective number of simulation draws
        ac = AutoCorrelation(draws, var_plus)
        neff = m * n / (1 + 2 * sum(ac))

        return R_hat, neff, phi_bar


    def MetropolisHastings(
        initialStates,
        numberOfDraws=1000,
        maxNumberOfIterations=10,
        warmup=None,
        show_print=False
    ):
        """Implements the Metropolis Hastings algorithm, checking for stationarity

        :param initialStates: list of inintial states for the sequences
        :type initialStates: list(state)

        :param numberOfDraws: numberOfDraws requested by the user
        :type numberOfDraws: int

        :param maxNumberOfIterations: Draws are generated at each
            iteration until stationarity is detected. This parameter sets
            a maximum number of these iterations.
        :type maxNumberOfIterations: int

        :return: the draws, the estimated average, the status of
            stationarity and the number of iterations
        :rtype: numpy.array, float, bool, int

        """
        # number of sequences = number of initial states
       
        indicator = False
        m = 2 * len(initialStates)
        
        iterators = [
            MetropolisHastingsIterator(init_state) for init_state in initialStates
        ]

        number_of_parallel_jobs = min(cpu_count(), len(iterators))

        # Warmup
        print(
            f'***** Warmup with {len(iterators)} sequences '
            f'of {numberOfDraws} draws *****'
        )
        # We first generate draws that are not stored for the warmup
        # of the Markov processes
        _ = Parallel(n_jobs=number_of_parallel_jobs, prefer='threads')(
            delayed(generate_draws)(i, numberOfDraws) for i in iterators
        )
        


        currentNumberOfDraws = numberOfDraws
        for trials in range(maxNumberOfIterations):
            print(
                f'***** Trial {trials} with {len(iterators)} sequences '
                f'of {currentNumberOfDraws} draws *****'
            )
            # Generate the draws
            draws = Parallel(n_jobs=number_of_parallel_jobs, prefer='threads')(
                delayed(generate_draws)(i, currentNumberOfDraws) for i in iterators
            )
            
            
        
            #it is necessary to omit array of individuals for convergancy analysis, it should be adapted also for individuals later on
           
            draws = np.array(draws[:])
        
        
            print(f'Generated draws: {draws.shape}')
            for i in iterators:
                print(f'Success rate: {i.getSuccessRate()}')

            # The dimensions of draws are: #sequences x #draws x #indicators
            # We change it to obtain: #indicators x #sequences x #draws

            draws = np.swapaxes(np.swapaxes(draws, 0, 2), 1, 2)
            
          
            draws_analyse = draws[[2,3]]
            R_hat, neff, phi_bar = zip(
                *[AnalyzeDraws(draw_per_indicator) for draw_per_indicator in draws_analyse]
            )
            R_hat = np.array(R_hat)
            neff = np.array(neff)
            target_neff = 5 * m
            phi_bar = np.array(phi_bar)

            print(f'Potential scale reduction: {R_hat}')
            print('    should be at most 1.1')
            print(f'Effective number of simulation draws: {neff}')
            print(f'    should be at least {target_neff}')
            if np.all(R_hat <= 1.1) and np.all(neff >= target_neff):
                # We merge the draws from the various sequences before
                # returning them
                final_draws = np.array([t.flatten() for t in draws])
                return final_draws, phi_bar, True, trials + 1
            min_neff = np.min(neff)
            if min_neff >= target_neff:
                inflate = 2
            else:
                inflate = 1 + int(target_neff / min_neff)
            currentNumberOfDraws = currentNumberOfDraws * inflate
        print(
            f'Warning: the maximum number ({maxNumberOfIterations}) '
            f'of iterations has been reached before stationarity.'
        )
        # We merge the draws from the various sequences before returning
        # them.
        final_draws = np.array([t.flatten() for t in draws])
        #the last one is one sequence
        return final_draws, phi_bar, False, maxNumberOfIterations

    #An implementation of State abstract class (you have to have state.py file for this)
    class distribution_draws(State):
        def __init__(
            self, 
            xy,
            
        ):
            """Constructor
            """
            self.xy = xy # [htype, hsize, nbcars, (total_num_licences), list of individuals]
        
        
        def indicators(self):
            """The indicators are the parameters of interest, generated 
            by each draw. In this case, the indicators are the states 
            themselves.

            :return: array of indicators
            :rtype: numpy.array()
            """
            # print(f'Draw:{self.xy}')
            #[htype,hsize,nbcars,individuals]
            return self.xy
        
        
        #this can be expanded to various number of dimensions
        def next_state(self):
            r = random.randint(0, 3) #num_dim-1
          
            if hsize_ref<3:
                while r==1:
                    r = random.randint(0, 3)

            if(r==0): #htype 
                
                hsize_value = self.xy[1]
                nbcars_value = self.xy[2]
                licences_value = self.xy[3]
                list_of_people = self.xy[4]
                htype_value = draw_type_model(hsize_value, list_of_people)
                if hsize_value > 2:
                    list_of_people = copy.deepcopy(self.xy[4][:2])
                    
                    licence_driving = []
                    for each in list_of_people:
                        if each[4]==1:
                            licence_driving.append(1)
                        else:
                            licence_driving.append(2)
                            
                    for i in range(3, hsize_value+1):
                        if i==3:
                            agent = [1,1,1,1,1]
                            agent[0] = draw_age_member(list_of_people[1][0], htype_value, list_of_people[0][0], list_of_people[1][0])
                            agent[1] = draw_gender_member(htype_value)
                            agent[2] = draw_marital_member(agent[0],htype_value)
                            agent[3] = draw_employment(agent[0])
                            agent[4] = draw_driving(agent[0], agent[4], htype_value, nbcars_value, licences=-1)
                        else:

                            agent = [1,1,1,1,1]
                            agent[0] = draw_age_member(list_of_people[i-2][0], htype_value)
                            agent[1] = draw_gender_member(htype_value)
                            agent[2] = draw_marital_member(agent[0],htype_value)
                            agent[3] = draw_employment(agent[0])
                            agent[4] = draw_driving(agent[0], agent[4], htype_value, nbcars_value, licences=-1)
                            
                        list_of_people.append(agent)
                    
                    for _ in range(hsize_value,17):
                        list_of_people.append([0,0,0,0,0])
                    
                    # licences
                    licences_value = 0
                    for each in list_of_people:
                        if each[4]==1:
                            licences_value += 1
                    
            elif(r==1): #draw hsize
                
                # hsize
                choice = random.choices(hsize.index,weights=hsize.values,k=1) #hszie is drawn from marginal distribution
                hsize_value = choice[0]
                htype_value = draw_type_model(hsize_value, self.xy[4])
                nbcars_value = self.xy[2]
                
                hsize_prev = np.min([self.xy[1],2])
                # list of people
                if hsize_value > hsize_prev: # number of people increase
                    list_of_people = copy.deepcopy(self.xy[4][:hsize_prev])
                    licence_driving = []
                    for each in list_of_people:
                        if each[4]==1:
                            licence_driving.append(1)
                        else:
                            licence_driving.append(2)
                    # generate individuals
                    for i in range(hsize_prev+1, hsize_value+1): 
                        if i == 2:
                            agent = [1,1,1,1,1]
                            agent[0] = draw_age_second(htype_value,self.xy[4][0][0])
                            agent[1] = draw_gender_second(htype_value, self.xy[4][0][1])
                            agent[2] = draw_marital_second(htype_value,agent[0],self.xy[4][0][2])
                            agent[3] = draw_employment(agent[0])
                            agent[4] = draw_driving_spouse(nbcars, X_curr[0], hsize_value, licence_driving)
                      
                        elif i==3:
            
                            agent = [1,1,1,1,1]
                            agent[0] = draw_age_member(list_of_people[1][0], htype_value, list_of_people[0][0], list_of_people[1][0])
                            agent[1] = draw_gender_member(htype_value)
                            agent[2] = draw_marital_member(agent[0],htype_value)
                            agent[3] = draw_employment(agent[0])
                            agent[4] = draw_driving(agent[0], agent[4], htype_value, nbcars_value, licences=-1)
                        else:

                            agent = [1,1,1,1,1]
                            agent[0] = draw_age_member(list_of_people[i-2][0], htype_value)
                            agent[1] = draw_gender_member(htype_value)
                            agent[2] = draw_marital_member(agent[0],htype_value)
                            agent[3] = draw_employment(agent[0])
                            agent[4] = draw_driving(agent[0], agent[4], htype_value, nbcars_value, licences=-1)
                            
                        list_of_people.append(agent)
                                            
                else:
                    list_of_people = copy.deepcopy(self.xy[4][:hsize_value])
                    
                for _ in range(hsize_value,17):
                    list_of_people.append([0,0,0,0,0])
                
                # licences
                licences_value = 0
                for each in list_of_people:
                    if each[4]==1:
                        licences_value += 1

                
            elif(r==2): # draw nbcars
                
                htype_value = self.xy[0]
                hsize_value = self.xy[1]
                licences_value = self.xy[3]
                nbcars_value = draw_from_nbcars_b(htype_value, hsize_value, licences_value)
                list_of_people = self.xy[4]           
                    
            else: #draw feature of people
                htype_value = self.xy[0] 
                hsize_value = self.xy[1]
                nbcars_value = self.xy[2]
                licences_value = self.xy[3]
                list_of_people = self.xy[4]
                
                licence_driving = []
                for each in list_of_people:
                    if each[4]==1:
                        licence_driving.append(1)
                    else:
                        licence_driving.append(2)
                
                p = random.randint(0, hsize_value - 1) # pick individual
                list_of_people = draw_feature_people(list_of_people,htype_value,nbcars_value,licences_value,p,licence_driving,hsize_value)
                
                licences_value = 0
                for each in list_of_people:
                    if each[4]==1:
                        licences_value += 1
                        
            
            next_state = distribution_draws(
                np.array([htype_value, hsize_value, nbcars_value,licences_value, list_of_people]),
                
        )
            
        
            return (
                next_state,
                0,
                0,
            )
        
        
        def logweight(self):
            return 0
    # Reading data
    numberOfDraws = (hh.hid.nunique()+1) // 2 * 2 #the part with the division and multiplication might be redundant
    maxNumberOfIterations = 5
    #it is important to choose some states that exist in dataset
    #the initial states are completely random
    if hsize_ref==1:
        initialStates = [
            distribution_draws(
                np.array([10,1,1,1,[[3,1,1,1,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),
            distribution_draws(
                np.array([10,1,1,1,[[4,2,1,1,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),
            distribution_draws(
                np.array([10,1,1,1,[[3,2,1,1,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),
            distribution_draws(
                np.array([10,1,1,1,[[5,1,1,4,1],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),

        ]

    elif hsize_ref==2:
        initialStates = [
            distribution_draws(
                np.array([210,2,1,1,[[4,1,2,1,1],[4,2,2,2,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
            
            ),  
            
            distribution_draws(
                np.array([210,2,1,2,[[5,2,1,1,1],[4,1,1,1,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
            
            ),
            distribution_draws(
                np.array([230,2,1,1,[[4,1,4,1,1],[1,2,1,3,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),
            distribution_draws(
                np.array([30,2,1,1,[[3,1,4,1,1],[3,2,1,3,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),
            
        ]

    else:
        initialStates = [
            distribution_draws(
                np.array([30,3,1,2,[[5,2,1,1,1],[4,1,1,1,1],[4,1,1,1,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
            
            ),
            distribution_draws(
                np.array([230,3,1,1,[[4,1,4,1,1],[1,2,1,3,2],[1,1,1,3,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),

            distribution_draws(
                np.array([220,3,2,2,[[4,1,2,1,1],[4,2,2,2,1],[1,1,1,3,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            ),
            distribution_draws(
                np.array([220,3,2,2,[[4,1,2,1,1],[4,2,2,2,1],[3,1,1,3,2],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]\
                                        ,[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]]), 
                
            )
            
        ]
    
    #Testing
    start_time = time.time()
    draws, estimates, convergence, numberOfTrials= MetropolisHastings(
        initialStates,
        numberOfDraws,
        maxNumberOfIterations,
    )
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time} seconds")
    
    def take_every_nth_element(draws, nb_dim,nbchain, start=0):
        data = []
        for i in range(nb_dim):
            data.append(draws[i][start::nbchain])
        res = pd.DataFrame(data) 
        return res.T

    def convert_a_row(ind, htype, nbcars, hsize,individuals,df):
        for i in range(len(individuals)):
            age = individuals[i][0]
            gender = individuals[i][1]
            marital = individuals[i][2]
            employment = individuals[i][3]
            licence = individuals[i][4]
            if(age!=0 and gender!=0 and marital!=0 and employment!=0 and licence!=0):
                data = {'hid':ind, 'htype':htype,'nbcars_agg':nbcars,'hsize':hsize,'age_discrete':age,'gender':gender,'marital_status':marital,'employment':employment,'driving_licence':licence}
                df.append(data)
    def extract_info_on_individuals(sample):
        '''
        Attributes of record: hid, htype, nbcars, hsize, age_discrete, gender, martial_status, employment
        '''
        arr = []
        result = [convert_a_row(i,row[0],row[1],row[2],row[3],arr) for i, row in enumerate(zip(sample['htype'],sample['nbcars'],sample['hsize'],sample['individuals']))]
        return arr
    sample= take_every_nth_element(draws,5,len(initialStates)) #number of chains is always equal to number of intitial_states
    sample.rename(columns = {0:'htype', 1:'hsize', 2:'nbcars', 3:'num_licences', 4:'individuals'}, inplace = True)
    result = pd.DataFrame(extract_info_on_individuals(sample))
    result.to_csv(path + f"/synthetic_households_size{hsize_ref}.csv",index=False)
    #randomly pick one individual in each househould
    result = result.groupby('hid').apply(lambda x: x.sample(1)).reset_index(drop=True)
    result.to_csv(path+ f"/synthetic_individuals_size{hsize_ref}.csv",index=False)
    
    
    B = 1 #change to the bigger number to do bootstrap
    start_time = datetime.datetime.now()
    for i in range(B):
         #Testing
        draws, _, _, _= MetropolisHastings(
            initialStates,
            numberOfDraws,
            maxNumberOfIterations=1,
            warmup = 100,
        )
        for j in range(len(initialStates)):
            pathbs = os.path.join(path, 'bs'+str(i) + '_try'+str(j))
            try : 
                os.mkdir(pathbs)
            except:
                pass
            
            end_time = datetime.datetime.now()
            sample= take_every_nth_element(draws,5,len(initialStates), start=j) #number of chains is always equal to number of intitial_states
            sample.rename(columns = {0:'htype', 1:'hsize', 2:'nbcars', 3:'num_licences', 4:'individuals'}, inplace = True)
            result_hh = pd.DataFrame(extract_info_on_individuals(sample))
            result_hh.to_csv(os.path.join(pathbs, "htype"+str(hsize_ref)+'_hh.csv'), index=False)
            execution_time = end_time - start_time
            print(f"Execution time: {execution_time} seconds")

***** Warmup with 4 sequences of 19372 draws *****
***** Trial 0 with 4 sequences of 19372 draws *****
Generated draws: (4, 19372, 5)
Success rate: 1.0
Success rate: 1.0
Success rate: 1.0
Success rate: 1.0
Potential scale reduction: [0.99994838 1.00177121]
    should be at most 1.1
Effective number of simulation draws: [15211.80061804  2397.33305507]
    should be at least 40
Execution time: 193.31590604782104 seconds
***** Warmup with 4 sequences of 19372 draws *****
***** Trial 0 with 4 sequences of 19372 draws *****
Generated draws: (4, 19372, 5)
Success rate: 1.0
Success rate: 1.0
Success rate: 1.0
Success rate: 1.0
Potential scale reduction: [0.99994838 1.00221188]
    should be at most 1.1
Effective number of simulation draws: [15090.59022589  2239.15887219]
    should be at least 40
Execution time: 0:03:58.460891 seconds
Execution time: 0:04:00.077306 seconds
Execution time: 0:04:01.722656 seconds
Execution time: 0:04:03.729560 seconds
***** Warmup with 4 sequences of 20206 draw

## Merge

In [ ]:
import os
# Merge synthetic data
# They are in results folder
# We merge the files containing the word "households"
# We merge the files containing the word "individuals"

path = "results/"

if os.path.exists(path + "synthetic_households.csv"):
    os.remove(path + "synthetic_households.csv")
if os.path.exists(path + "synthetic_individuals.csv"):
    os.remove(path + "synthetic_individuals.csv")

last_id_hh = 0
last_id_indiv = 0
for f in os.listdir(path):
    if "households" in f:
        df = pd.read_csv(path + f)
        df["hid"] += last_id_hh
        last_id_hh = df["hid"].max() + 1
        df.to_csv(path + "synthetic_households.csv", mode='a', 
                  index=False, header= not os.path.exists(path + "synthetic_households.csv"))
    if "individuals" in f:
        df = pd.read_csv(path + f)
        df["hid"] += last_id_indiv
        last_id_indiv = df["hid"].max() + 1
        df.to_csv(path + "synthetic_individuals.csv", mode='a',
                  index=False, header= not os.path.exists(path + "synthetic_individuals.csv"))

### Merge for bootstrap

In [428]:
import os
# Merge synthetic data
# They are in results folder
# We merge the files containing the word "households"
# We merge the files containing the word "individuals"

# Create dir in path
path = "results/"
result_path = path + "/merged/"
try :
    os.mkdir(result_path)
except:
    pass

# empty results folder
for f in os.listdir(result_path):
    os.remove(result_path + f)

# From results folder, get last bsid
last_bsid = 0
for f in os.listdir(result_path):
    last_bsid = max(last_bsid, int(f.split("_")[1]))
last_bsid +=1

for i, folder in enumerate(os.listdir(path)):
    if "bs" not in folder:
        continue
    folder_path = path + folder + "/"
    last_id_hh = 0
    last_id_indiv = 0

    for f in os.listdir(folder_path):
        df = pd.read_csv(folder_path + f)
        df["hid"] += last_id_hh
        last_id_hh = df["hid"].max() + 1
        name_file = f"bs_{i+last_bsid}_hh.csv"
        df.to_csv(result_path + name_file, mode='a', 
                index=False, header= not os.path.exists(result_path + name_file))
